In [81]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import torch
from fastai.lm_rnn import *

In [70]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/rossmann/'

In [114]:
joined = pd.read_feather(f'{PATH}joined')
joined_test = pd.read_feather(f'{PATH}joined_test')

In [115]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen',
    'Promo2Weeks', 'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear',
    'State', 'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw',
    'SchoolHoliday_fw', 'SchoolHoliday_bw']

contin_vars = ['CompetitionDistance', 'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

n = len(joined); n

844338

In [116]:
dep = 'Sales'
joined = joined[cat_vars+contin_vars+[dep, 'Date']].copy()

In [117]:
joined_test[dep] = 0
joined_test = joined_test[cat_vars+contin_vars+[dep, 'Date', 'Id']].copy()

In [118]:
for v in cat_vars: joined[v] = joined[v].astype('category').cat.as_ordered()

In [119]:
apply_cats(joined_test, joined)

In [120]:
for v in contin_vars:
    joined[v] = joined[v].fillna(0).astype('float32')
    joined_test[v] = joined_test[v].fillna(0).astype('float32')

In [111]:
idxs = get_cv_idxs(n, val_pct=150000/n)
joined_samp = joined.iloc[idxs].set_index("Date")

In [121]:
samp_size = n
joined_samp = joined.set_index("Date")

In [122]:
df, y, nas, mapper = proc_df(joined_samp, 'Sales', do_scale=True)
yl = np.log(y)

In [123]:
joined_test = joined_test.set_index("Date")

In [124]:
df_test, _, nas, mapper = proc_df(joined_test, 'Sales', do_scale=True, skip_flds=['Id'],
                                  mapper=mapper, na_dict=nas)

In [125]:
val_idx = np.flatnonzero(
    (df.index<=datetime.datetime(2014,9,17)) & (df.index>=datetime.datetime(2014,8,1)))

In [22]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

In [126]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, yl.astype(np.float32), cat_flds=cat_vars, bs=512,
                                       test_df=df_test)

In [127]:
cat_sz = [(c, len(joined_samp[c].cat.categories)+1) for c in cat_vars]

In [128]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [129]:
sum([outp for inp, outp in emb_szs]) + 16

215

In [130]:
class RNN(nn.Module):
    def __init__(self, emb_sizes, n_hidden, conts, bs, n_layers, seq_len, p1, p2):
        super().__init__()
        self.n_layers = n_layers
        self.seq_len = seq_len
        self.e = nn.ModuleList([nn.Embedding(emb[0], emb[1]) for emb in emb_sizes])
        self.n_RNN = sum([outp for inp, outp in emb_sizes]) + conts
        self.drop1 = nn.Dropout(p1)
        self.bs = int(bs/seq_len)
        self.init_bn(self.bs)
        self.rnn = nn.RNN(self.n_RNN, n_hidden)
        self.drop2 = nn.Dropout(p2)
        self.L_out = nn.Linear(n_hidden, 1)
        self.init_hidden(self.bs)
        
    def forward(self, cats, conts):
        bs = cats.size(0)
        n_cats = cats.size(1)
        cat_emb = []
        for i, emb in enumerate(self.e):
            qwert = emb(cats[:,i])
            cat_emb.append(qwert)
        inp = torch.cat(cat_emb, dim=1)
        inp = torch.cat([inp, conts], dim=1)
        self.bs = int(np.ceil(bs/self.seq_len))
        inp = self.drop1(inp.view(seq_len, self.bs, self.n_RNN))
        if self.h.size(1) != self.bs: 
            self.init_hidden(self.bs)
        outp, h = self.rnn(inp, self.h)
        self.h = repackage_var(h)
        out2 = self.L_out(self.drop2(outp))
        return out2.view(bs, 1)
                           
    def init_hidden(self, bs):
        self.h = V(torch.zeros(self.n_layers, bs, n_hidden))
        
    def init_bn(self, bs):
        self.bn = nn.BatchNorm1d(bs)

In [131]:
n_hidden = 256
bs = 512
n_layers = 1
conts = 16
seq_len = 8
p1 = 0.001
p2 = 0.01
model_RNN = RNN(emb_szs, n_hidden, conts, bs, n_layers, seq_len, p1, p2)

In [132]:
lr = 1e-3
opt = optim.Adam(model_RNN.parameters(), lr)

In [133]:
learner = Learner.from_model_data(model_RNN, md)

In [134]:
learner.fit(lr, 3, cycle_len=1)

100%|█████████▉| 1570/1575 [00:21<00:00, 77.50it/s, loss=0.156]

RuntimeError: invalid argument 2: size '[8 x 7 x 215]' is invalid for input with 10965 elements at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/TH/THStorage.c:41


 0.2839  2.1820  1.9079  ...  -1.2011 -0.8979 -0.4899
-0.6258  0.6559  0.5317  ...   0.8856 -0.8979  2.0410
-0.6386  0.8907  0.5317  ...  -1.7228  1.1137 -0.4899
          ...             ⋱             ...          
 0.5415  0.1864  0.1189  ...   0.6094 -0.8979 -0.4899
-0.6745  0.7733  0.2565  ...   0.8242 -0.8979 -0.4899
-0.2529  0.7733  0.9446  ...   1.1311  1.1137  2.0410
[torch.cuda.FloatTensor of size 128x16 (GPU 0)]